In [ ]:
# what'sgoing on...

# XAMPP can only upload a file of 40mb at a time and it's not grwat at parsing the data
# gotta clean it in python and then upload with sqlalchemy
# release date column is a string, need to format it correctly in python before inserting


In [1]:
# import libraries

import numpy as np
from math import pi
import pandas as pd
from sqlalchemy import create_engine, Column, select, Integer, Text, MetaData, Table, String, Date, ForeignKey, DateTime, Float

pd.set_option('display.max_columns', False)
pd.set_option('display.max_rows', False)


engine = create_engine('mysql://root:@localhost/movies_db', echo=False)
metadata = MetaData()

In [2]:
# Load the datasets

yup = pd.read_csv(r'C:\Users\Brian\Desktop\DS Projects\movies_sql_project\AllMoviesDetailsCleaned.csv', sep = ';')

okay = pd.read_csv(r'C:\Users\Brian\Desktop\DS Projects\movies_sql_project\AllMoviesCastingRaw.csv', sep = ';')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
cols = ['ID', 'Budget', 'Genres', 'IMDB_ID', 'Original_Language',
       'Original_Title', 'Overview', 'Popularity', 'Production_Companies',
       'Production_Countries', 'Release_Date', 'Revenue', 'Runtime',
       'Spoken_Languages', 'Status', 'Tagline', 'Title', 'Vote_Average',
       'Vote_Count', 'Production_Companies_Count', 'Production_Countries_Count',
       'Spoken_Languages_Count']

yup.columns = cols

In [5]:
# change the release_date column format

yup['Release_Date'] = pd.to_datetime(yup['Release_Date'], dayfirst = True)

In [32]:
# let's check the min/max lengths of the string columns in our table before limiting its sizes

print(yup['Genres'].str.len().min())
print(yup['Genres'].str.len().max())

print(yup['IMDB_ID'].str.len().min())
print(yup['IMDB_ID'].str.len().max())

print(yup['Original_Language'].str.len().min())
print(yup['Original_Language'].str.len().max())

print(yup['Original_Title'].str.len().min())
print(yup['Original_Title'].str.len().max())

print(yup['Overview'].str.len().min())
print(yup['Overview'].str.len().max())

print(yup['Production_Companies'].str.len().min())
print(yup['Production_Companies'].str.len().max())

print(yup['Production_Countries'].str.len().min())
print(yup['Production_Countries'].str.len().max())

print(yup['Spoken_Languages'].str.len().min())
print(yup['Spoken_Languages'].str.len().max())

print(yup['Status'].str.len().min())
print(yup['Status'].str.len().max())

print(yup['Tagline'].str.len().min())
print(yup['Tagline'].str.len().max())

print(yup['Title'].str.len().min())
print(yup['Title'].str.len().max())

3.0
90.0
8.0
10.0
2.0
2.0
1.0
207.0
1.0
1000.0
2
141
4
44
3.0
16.0
7.0
15.0
1.0
954.0
1.0
212.0


In [12]:
movie_details = Table(
    'movie_details', metadata,
    Column("ID", Integer, primary_key = True),
    Column("Budget", Integer),
    Column("Genres", String(100)),
    Column("IMDB_ID", String(12)),
    Column("Original_Language", String(2)),
    Column("Original_Title", String(210)),
    Column("Overview", String(1000)),
    Column("Popularity", Float),
    Column("Production_Companies", String(150)),
    Column("Production_Countries", String(50)),
    Column("Release_Date", Date),
    Column("Revenue", Integer),
    Column("Runtime", Float),
    Column("Spoken_Languages", String(20)),
    Column("Status", String(20)),
    Column("Tagline", String(1000)),
    Column("Title", String(250)),
    Column("Vote_Average", Float),
    Column("Vote_Count", Integer),
    Column("Production_Companies_Count", Integer),
    Column("Production_Countries_Count", Integer),
    Column("Spoken_Languages_Count", Integer),
)

movie_details.create(bind=engine)

In [16]:
# load the movie_details dataset into MySQL database

yup.to_sql('movie_details', con=engine, if_exists='append', index=False)

In [17]:
# Load the casting dataset into the database

okay.to_sql('movie_casting', con=engine, if_exists='replace', index=False)

# Let's answer some business questions with SQL


In [ ]:
# How many 'big-budget' movies are there? (> 100,000,000)
# How many 'highly-rated' movies? (> 8.0)
# How many movies have a low sample size of reviews? (< 20)
# How many movies have no tagline? (Null value count)
# How many produced in the US?
# Let's make a continents/international column?
# How many countries have produced movies? (distinct values)
# 20th century vs 21st century movies
# How many small-budget movies have more than 2-3 production companies?
# How many movies retained their original titles? 
# average run-time
# avg cost of movies w/ reported budgets? (>1)


In [ ]:
# How many 'big-budget' movies are there? (> 100,000,000)

SELECT * FROM movie_details
WHERE budget > 100000000;

# Results in 342 records

In [ ]:
# How many 'highly-rated' movies? (> 8.0)

SELECT * FROM movie_details
WHERE vote_average > 8.0;

# Results in 11,160 records

In [ ]:
# How many movies have a low sample size of reviews? (< 20)

SELECT * FROM movie_details
WHERE vote_count < 20;

# Results in 313,489 records

In [ ]:
# How many movies have no tagline? (Null value count) [276,113 hits]

SELECT count(*) FROM `movie_details`
WHERE tagline IS NULL;

# Results in 276,133 records

In [ ]:
# How many produced in the US?

#first, gotta check the distinct values under production_countries

SELECT DISTINCT production_countries FROM `movie_details`;

#okay, United States has two options: United States of America and United States Minor Outlying Islands

SELECT count(*) FROM `movie_details` 
WHERE Production_Countries = 'United States of America' OR 'United States Minor Outlying Islands';

# Results in 55,504 records

In [ ]:
# Let's make a continents/international column?

ALTER TABLE `movie_details` ADD COLUMN Domestic char(3);

#created, now I need to fill it with 'yes' or 'no'

UPDATE `movie_details` SET Domestic = 'Yes' WHERE Production_Countries = 'United States of America' OR 'United States Outlying Islands';

UPDATE `movie_details` SET Domestic = 'No' WHERE Production_Countries != 'United States of America' OR 'United States Outlying Islands';


In [ ]:
# How many countries have produced movies? (distinct values)

SELECT COUNT(DISTINCT(Production_Countries)) FROM `movie_details`;

# There are 235 different countries

In [ ]:
# how to find the next 5 oldest movies after the first 10

SELECT * FROM `movie_details`
WHERE Release_Date IS NOT NULL
ORDER BY Release_Date
LIMIT 5 OFFSET 10;

In [ ]:
# How many small-budget movies have more than 2-3 production companies?

SELECT count(*) FROM `movie_details`
WHERE( 
	budget BETWEEN 1 AND 50000
)
AND
(
    Production_Companies_Count >= 2
)
;

# Results in 517 records

In [ ]:
# How many movies retained their original titles? 

SELECT count(*) FROM `movie_details`
WHERE Original_Title = Title
;

# Results in 267,431 records

In [ ]:
# average run-time

SELECT ROUND(AVG(CAST(Runtime AS FLOAT)), 2) FROM `movie_details`;

# The average is 60.41 minutes

In [ ]:
# avg cost of movies w/ reported budgets? (>1)

SELECT AVG(Budget) FROM `movie_details`
WHERE Budget >= 1;

# The average is 12,779,511.82 USD

In [ ]:
# who was the main actor for the biggest flop?

SELECT details.Title, details.Budget, details.Revenue, details.Budget, cast.actor1_name
FROM movie_details details
JOIN movie_casting cast
	on details.ID = cast.id
WHERE details.budget > 100000000 and details.Revenue BETWEEN 1 AND 50000000
order by 1;

In [ ]:
# top 10 actors by appearance with a temp table / view

DROP TABLE IF EXISTS actor_appearances;

CREATE TABLE actor_appearances(
    id bigint(20),
    Title varchar(255),
    `actor1_name` varchar(255),
    `actor2_name` varchar(255),
    `actor3_name` varchar(255),
    `actor4_name` varchar(255),
    `actor5_name` varchar(255)
);


INSERT INTO actor_appearances
SELECT cast.id, details.Title, cast.actor1_name, cast.actor2_name, cast.actor3_name, cast.actor4_name, cast.actor5_name
FROM movie_casting AS cast
LEFT JOIN movie_details AS details
	ON cast.id = details.ID;
    
SELECT actor1_name, COUNT(*) AS credits FROM actor_appearances
WHERE actor1_name != 'None'
GROUP BY actor1_name
ORDER BY credits DESC LIMIT 10;

# Business Questions Ideas


In [ ]:
# total number of english-speaking movies as %
SELECT COUNT(CASE Original_Language WHEN 'en' then 1 else null end) as ENG_TOTAL, (COUNT(CASE Original_Language WHEN 'en' then 1 else null end)/COUNT(*)) * 100 as ENG_PCT
FROM movie_details;

In [ ]:
# for each country, which movie had the highest revenue after budget

SELECT Production_Countries, Title, Budget, MAX(Revenue)
FROM movie_details
WHERE Revenue > 0 AND Budget > 0
GROUP BY 1
ORDER BY 4 DESC;

In [ ]:
# Count for the number of boys, girls, and NTA in the first ten rows of movie_casting table

WITH actor_genders AS 
(
    SELECT id, actor1_gender, actor2_gender, actor3_gender, actor4_gender, actor5_gender
    FROM `movie_casting`
    WHERE actor1_name != 'None'
    ORDER BY id
    LIMIT 10
)

SELECT
SUM(NTA_1 + NTA_2 + NTA_3 + NTA_4 + NTA_5) AS NTA_Total,
SUM(Girls_1 + Girls_2 + Girls_3 + Girls_4 + Girls_5) AS Girls_Total,
SUM(Boys_1 + Boys_2 + Boys_3 + Boys_4 + Boys_5) AS Boys_Total
FROM
(
	SELECT
    COUNT(CASE WHEN actor1_gender = 0 THEN actor1_gender END) AS NTA_1,
	COUNT(CASE WHEN actor1_gender = 1 THEN actor1_gender END) AS Girls_1,
	COUNT(CASE WHEN actor1_gender = 2 THEN actor1_gender END) AS Boys_1,
	COUNT(CASE WHEN actor2_gender = 0 THEN actor2_gender END) AS NTA_2,
	COUNT(CASE WHEN actor2_gender = 1 THEN actor2_gender END) AS Girls_2,
	COUNT(CASE WHEN actor2_gender = 2 THEN actor2_gender END) AS Boys_2,
	COUNT(CASE WHEN actor3_gender = 0 THEN actor3_gender END) AS NTA_3,
	COUNT(CASE WHEN actor3_gender = 1 THEN actor3_gender END) AS Girls_3,
	COUNT(CASE WHEN actor3_gender = 2 THEN actor3_gender END) AS Boys_3,
	COUNT(CASE WHEN actor4_gender = 0 THEN actor4_gender END) AS NTA_4,
	COUNT(CASE WHEN actor4_gender = 1 THEN actor4_gender END) AS Girls_4,
	COUNT(CASE WHEN actor4_gender = 2 THEN actor4_gender END) AS Boys_4,
	COUNT(CASE WHEN actor5_gender = 0 THEN actor5_gender END) AS NTA_5,
	COUNT(CASE WHEN actor5_gender = 1 THEN actor5_gender END) AS Girls_5,
	COUNT(CASE WHEN actor5_gender = 2 THEN actor5_gender END) AS Boys_5
	FROM actor_genders
) as subquery;

In [ ]:
# running count of 15 highest revenue films in France

SELECT Title, Production_Countries, ID, Revenue, SUM(Revenue) OVER(ORDER BY Revenue DESC) AS Rolling_Revenue
FROM movie_details
WHERE Revenue > 0 and Production_Countries = 'France'
ORDER BY 4 DESC
LIMIT 15;